In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from pytube import YouTube
from pymongo import MongoClient
from tqdm import tqdm
import time
import datetime

In [18]:
# MongoDB 클라이언트 및 컬렉션 설정
client = MongoClient(host="", port=, username='', password='')
db = client['YouTube_crawling']
collection = db['OneAsia_YouTube_crawling']

In [19]:
# 웹드라이버 초기화 및 YouTube 검색 페이지 열기
driver = webdriver.Chrome()
driver.get("https://www.youtube.com/")

In [20]:
# 검색어 입력
elem = driver.find_element(By.NAME, "search_query")
elem.send_keys("釜山ワンアジアフェスティバル")
elem.send_keys(Keys.RETURN)
time.sleep(3)

In [21]:
# 동영상 필터 버튼을 찾을 때까지 최대 10초간 대기
video_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//div[@id='container' or @id='scroll-container']//yt-chip-cloud-chip-renderer//yt-formatted-string[contains(text(), '동영상')]"))
)

# 동영상 필터 버튼 클릭
video_button.click()

time.sleep(3)

In [22]:
# 맨 아래까지 스크롤
SCROLL_PAUSE_TIME = 1.5
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.documentElement.scrollHeight;")
    if new_height == last_height:
        break
    last_height = new_height

In [23]:
# 데이터 수집 및 MongoDB에 적재
titles = driver.find_elements(By.CSS_SELECTOR, "#dismissible.style-scope.ytd-video-renderer")

total_videos = len(titles)

for title in tqdm(titles, desc="동영상 수집 진행 중"):
    main_title = title.find_element(By.CSS_SELECTOR, "#video-title").get_property("title")
    tube_url = title.find_element(By.CSS_SELECTOR, "#video-title").get_property("href")
    aria_label = title.find_element(By.CSS_SELECTOR, "#video-title").get_attribute("aria-label")
    start_index = aria_label.find("게시자:") + len("게시자:")
    end_index = aria_label.find("조회수")
    channel_name = aria_label[start_index:end_index].strip()

    # 동영상 정보 수집
    tube = YouTube(tube_url)
    view_count = tube.vid_info.get("videoDetails", {}).get("viewCount")
    view = int(view_count) if view_count is not None else 0
    update_dates = str(tube.publish_date)
    update_date = update_dates.split(" ")
    length_second = int(tube.length)
    length = str(datetime.timedelta(seconds=length_second))

    # MongoDB에 데이터 적재
    data = {
        "제목": main_title,
        "URL": tube_url,
        "채널명": channel_name,
        "조회수": view,
        "업로드 날짜": update_date[0],
        "영상 길이": length,
        "검색키워드": "釜山ワンアジアフェスティバル"  # 검색 키워드를 지정 
    }
    collection.insert_one(data)

client.close()

동영상 수집 진행 중: 100%|██████████| 527/527 [12:48<00:00,  1.46s/it]


In [24]:
# 웹드라이버 종료
driver.quit()